In [ ]:
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.compat.v1.Session(config=config)
set_session(sess)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5



In [ ]:
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Embedding,Reshape,Activation,Input
from keras.layers.merge import Dot
from keras.layers import Input
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import skipgrams
from bs4 import BeautifulSoup
import gensim
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from string import punctuation

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
remove_terms = punctuation + "0123456789"

In [ ]:
def preprocessing(text):
    words = word_tokenize(text)
    tokens = [w for w in words if w.lower() not in remove_terms]
    #stopw = stopwords.words('english')
    #tokens = [token for token in tokens if token not in stopw]
    #remove words less than three letters
    #tokens = [word for word in tokens if len =(word)>=3]
    #remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    #lemmatize
    lemma = WordNetLemmatizer()
    tokens = [lemma.lemmatize(word) for word in tokens]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

In [ ]:
corpus = open('/content/stock_sentiment.csv', encoding="utf8").readlines()

In [ ]:
corpus = [preprocessing(sentence) for sentence in corpus if sentence.strip() !='']
corpus

['Text Sentiment',
 'Kickers on my watchlist XIDE TIT SOQ PNK CPW BPZ AJ trade method or method see prev post',
 'user AAP MOVIE return for the indicator just trade for the year AWESOME',
 'user I be afraid to short AMZN they are looking like a in eBooks and',
 'MNTA Over',
 'OI Over',
 'PGNX Over',
 'AAP user if so then the current downtrend will break Otherwise just a correction in',
 'Monday relative weakness NYX WIN TIE TAP ICE INT BMC AON C CHK BIIB',
 'GOOG ower trend line channel test volume support',
 'AAP will watch tomorrow for ONG',
 'i assuming FCX open tomorrow above the trigger buy still very much like this setup',
 'It really worry me how everyone expects the market to rally now usually exact opposite happens every time we shall see soon bac spx jpm',
 'AAP GAMCO arry Haverty Apple Is Extremely Cheap Great Video',
 'user Maykiljil posted that I agree that MSFT is going higher possibly north of',
 'Momentum is coming back to ETFC Broke resistance on solid volume Friday on

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

In [ ]:
X_train_tokens = tokenizer.texts_to_sequences(corpus)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

7465

In [ ]:
items = tokenizer.word_index.items()

In [ ]:
dim_embeddings=300
#inputs
inputs = Input(shape=(1, ), dtype='int32')
w=Embedding(vocab_size, dim_embeddings)(inputs)
#context
c_inputs = Input(shape=(1, ), dtype='int32')
c= Embedding(vocab_size, dim_embeddings)(c_inputs)

d= Dot(axes=2)([w,c])
d=Reshape((1,), input_shape=(1,1))(d)
d= Activation('sigmoid')(d)

model = Model(inputs=[inputs, c_inputs], outputs=d)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 300)       2239500     input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1, 300)       2239500     input_4[0][0]                    
____________________________________________________________________________________________

In [ ]:
model.compile(loss ='binary_crossentropy', optimizer='adam')

In [ ]:
n_epochs = 5
for epoch in range(n_epochs):
    loss =0;
    for i, doc in enumerate(X_train_tokens):
        data, labels = skipgrams(sequence =doc, vocabulary_size=vocab_size, window_size=4)
        x= [np.array(x) for x in zip(*data)]
        y=np.array(labels,dtype=np.int32)
        if x:
            loss += model.train_on_batch(x,y)
    
    print ('Epoch:', epoch, '\tloss:', loss)

NameError: ignored

In [ ]:
f = open('/content/stock1.csv', 'w' ,encoding="utf8")
f.write('{} {}\n'.format(vocab_size-1, dim_embeddings))
weights = model.get_weights()[0]
for word, i in items:
    f.write('{} {}\n'.format(word, ' '.join(map(str, list(weights[i,:])))))
f.close()

In [ ]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('/content/stock_sentiment.csv', binary=False)

ValueError: ignored

In [ ]:
w2v.most_similar(positive=['ibm']) 

NameError: ignored

In [ ]:
w2v.most_similar(positive=['cash'])

NameError: ignored